# 🧹 Tratamento e Transformação de Tabelas
## 📌 Visão Geral

### Este notebook tem como objetivo padronizar, tratar e transformar os datasets que compõem o projeto de crédito.
### O processo é orientado pelas melhores práticas de Engenharia de Dados, garantindo qualidade, confiabilidade e rastreabilidade para suportar análises avançadas e modelos preditivos.

## 🎯 Objetivos

### Construção da Tabela Base
Estruturar e validar a tabela principal (train_base), assegurando a unicidade da chave primária, tratamento de valores nulos e conversão de tipos de dados.

### Integração e Tratamento de Tabelas Satélites
Aplicar funções padronizadas de avaliação, transformação e criação de variáveis derivadas, permitindo consistência na manipulação de diferentes datasets.

### Documentação Automatizada de Variáveis
Atualizar o dicionário de dados com descrições em português, assegurando maior clareza, interpretabilidade e alinhamento entre equipes de negócio e tecnologia.

## 🛠️ Benefícios

Confiabilidade dos Dados: identificação e tratamento de inconsistências, duplicidades e nulos.

Escalabilidade: uso de funções reutilizáveis para diferentes tabelas, reduzindo retrabalho e aumentando eficiência.

Transparência: documentação clara das regras aplicadas e exportação de relatórios para consulta.

Performance: persistência das tabelas tratadas em formato Parquet, otimizando consumo em pipelines analíticos.

## ✅ Entregáveis

Tabela base tratada e consolidada.

Tabelas satélites padronizadas, enriquecidas e integradas.

Dicionário de variáveis atualizado, em formato legível para consulta por stakeholders técnicos e de negócio.

In [1]:
# Criação da SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploSparkSession") \
    .getOrCreate()

In [2]:
# Importações
from pyspark.sql.functions import col, when, count, isnan, isnull, countDistinct, to_date, format_string, to_date, datediff, lit, coalesce, isnan
from pyspark.sql.types import FloatType, DoubleType
from datetime import datetime
import pandas as pd
import os
import glob

In [56]:
caminho = r"C:\Users\fred\meu_projeto_etl"

### 📌 Função `avaliar_dataset(df, pk=None)`

Essa função tem como objetivo **avaliar a qualidade e as características de um DataFrame Spark**.  
Ela gera um diagnóstico útil para entender a estrutura, a presença de valores nulos e a consistência dos dados.  

---

#### 🔎 O que a função faz:

1. **Resumo inicial**
   - Conta o número total de registros do DataFrame.  
   - Obtém o schema (nome e tipo de cada coluna).  

2. **Análise coluna a coluna**  
   Para cada coluna do DataFrame, calcula:  
   - **Tipo de dado** (ex.: string, integer, double).  
   - **% de valores nulos** e a **quantidade absoluta de nulos**.  
     - Para colunas numéricas (`float`, `double`), também verifica valores `NaN`.  
   - **Quantidade de valores distintos** (cardinalidade da coluna).  
   - **Exemplo de até 3 valores distintos não nulos** (útil para inspecionar rapidamente o conteúdo).  
   - Campo reservado para **observações** (pode ser usado futuramente para anotações manuais).  

   👉 Todos esses resultados são consolidados em um **DataFrame Pandas**, facilitando a visualização tabular no notebook.

3. **Resumo geral do dataset**  
   Retorna um dicionário com:  
   - Total de registros.  
   - Quantidade de colunas que possuem valores nulos.  
   - Quantidade de colunas totalmente nulas (100% nulos).  
   - Número de duplicatas na chave primária (se informada).  

4. **Verificação de chave primária (opcional)**  
   - Se o parâmetro `pk` for passado e a coluna existir:  
     - Conta o número de duplicatas na coluna que deveria ser única.  
     - Informa no console se a chave é válida (sem duplicatas) ou se há problemas.  
   - Caso a coluna informada não exista no DataFrame, emite um alerta.  

---

#### ✅ Exemplos de uso

```python
# Avaliação básica
resultados, resumo = avaliar_dataset(df)

# Avaliação com verificação de chave primária
resultados, resumo = avaliar_dataset(df, pk="id_cliente")

# Exibir resultados
display(resultados)  # tabela detalhada por coluna
print(resumo)        # resumo geral


In [4]:
def avaliar_dataset(df, pk=None):

    print("📊 Avaliação do DataFrame...")
    total_registros = df.count()
    schema = dict(df.dtypes)
    resultados = []

    for c in df.columns:
        tipo = schema[c]
        cond = isnull(col(c))
        if tipo in ("float", "double"):
            cond = cond | isnan(col(c))

        nulos = df.filter(cond).count()
        distintos = df.select(c).distinct().count()

        # Captura de exemplos distintos e não nulos
        try:
            exemplos_df = df.select(c).where(col(c).isNotNull()).distinct().limit(3)
            exemplo = exemplos_df.toPandas()[c].tolist()
        except:
            exemplo = []

        resultados.append({
            "coluna": c,
            "tipo": tipo,
            "% nulos": round(nulos / total_registros * 100, 2),
            "qtd_nulos": nulos,
            "valores_distintos": distintos,
            "exemplo_valores": exemplo,
            "observacoes": ""
        })

    resultados_df = pd.DataFrame(resultados)

    # 🔍 Resumo geral
    resumo = {
        "total_registros": total_registros,
        "colunas_com_nulos": resultados_df[resultados_df["qtd_nulos"] > 0].shape[0],
        "colunas_totalmente_nulas": resultados_df[resultados_df["qtd_nulos"] == total_registros].shape[0],
        "duplicatas_pk": None
    }

    # Verificação de chave primária
    if pk and pk in df.columns:
        duplicatas_pk = df.groupBy(pk).count().filter(col("count") > 1).count()
        resumo["duplicatas_pk"] = duplicatas_pk
        print(f"\n🔍 Chave primária '{pk}':")
        if duplicatas_pk > 0:
            print(f"⚠️ Encontradas {duplicatas_pk} duplicatas com base na chave '{pk}'")
        else:
            print("✅ Chave primária é única.")
    elif pk:
        print(f"⚠️ A coluna '{pk}' não está presente no DataFrame.")


    return resultados_df, resumo



### 📌 Função `criar_flags_nulos(df)`

Essa função tem como objetivo **criar colunas auxiliares ("flags") que indicam a presença de valores nulos ou `NaN` em cada coluna de um DataFrame Spark**.  

---

#### 🔎 O que a função faz:

1. **Iteração sobre as colunas do DataFrame**  
   Para cada coluna:
   - Define uma condição `cond` que identifica registros onde o valor é **nulo** (`isNull`) ou **NaN** (`isnan`).  

2. **Verificação de existência de valores ausentes**  
   - Se a coluna tiver ao menos um valor ausente:  
     - Cria uma **nova coluna flag** com o sufixo `_flag`.  
       - Valor `1` → quando o registro é nulo/NaN.  
       - Valor `0` → quando o registro é válido.  
     - Exemplo: se a coluna original for `idade`, será criada `idade_flag`.  
   - Caso não haja valores ausentes:  
     - Nenhuma coluna extra é criada.  
     - Um aviso é impresso no console informando que a coluna está 100% completa.  

3. **Retorno**  
   - O DataFrame atualizado com as colunas de flags adicionadas.  

---

#### ✅ Exemplo de uso

```python
# Executar a função
df_flags = criar_flags_nulos(df)

# Listar todas as flags criadas
flags_criadas = [c for c in df_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
df_flags.select(flags_criadas).limit(5).toPandas()



In [5]:
def criar_flags_nulos(df):
    for c in df.columns:
        # Define a condição de ausência (null ou NaN)
        cond = col(c).isNull() | isnan(col(c))

        # Verifica se existe ao menos um valor ausente
        if df.filter(cond).count() > 0:
            nova_coluna = f"{c}_flag"
            df = df.withColumn(nova_coluna, when(cond, 1).otherwise(0))
            print(f"✅ Flag criada: {nova_coluna}")
        else:
            print(f"🟩 Sem valores ausentes em: {c} (flag não criada)")
    return df


### 📌 Função `detectar_e_converter_datas(df, formatos_tentados=None, max_amostras=None)`

Esta função tem como objetivo **detectar automaticamente colunas do tipo string que parecem representar datas** e convertê-las para o tipo `date` do PySpark.  
Ela é útil em datasets que chegam com campos de datas em formato textual e podem variar entre diferentes padrões (ex.: `yyyy-MM-dd`, `dd/MM/yyyy`, etc.).

---

#### 🔎 O que a função faz

1. **Parâmetros**
   - `df`: DataFrame Spark de entrada.
   - `formatos_tentados`: lista de formatos de data aceitos.  
     - Se não informado, usa um conjunto padrão que cobre formatos comuns no Brasil e no exterior.
   - `max_amostras`: número máximo de registros não nulos usados como amostra por coluna (default: 100).  
     - Evita trazer todo o dataset para o driver.

2. **Pré-processamento**
   - Considera apenas colunas do tipo `string`.  
   - Coleta até `max_amostras` valores não nulos da coluna, converte para string, remove espaços extras e transforma em lista.  
   - Se o valor contém horário (`2024-01-01 12:00:00` ou `2024-01-01T12:00:00`), apenas a parte da data é considerada.

3. **Validação de formatos**
   - Para cada formato da lista:
     - Tenta converter as amostras para `datetime` no Python (fazendo o mapeamento de tokens `yyyy → %Y`, `MM → %m`, `dd → %d`).  
     - Caso especial: formatos `yyyy-MM` são completados com `-01` (assumindo o primeiro dia do mês) apenas para validação.
     - Conta quantos valores foram parseados com sucesso e estão dentro de um intervalo plausível (anos entre 1950 e 2050).

4. **Critério de conversão**
   - Se **≥70% das amostras** de uma coluna forem válidas em algum formato, a função assume que esse é o padrão da coluna.  
   - A coluna é convertida no Spark com `to_date(coluna, formato)`.  
   - O nome da coluna é mantido (substituindo os valores originais).  
   - O processo é interrompido após o primeiro formato válido.

5. **Retorno**
   - Um novo DataFrame com as colunas convertidas para `date`.  
   - Uma lista com os nomes das colunas que foram convertidas.

---

#### ✅ Exemplo de uso

```python
df_convertido, colunas = detectar_e_converter_datas(df)

print("Colunas convertidas:", colunas)
df_convertido.printSchema()

# Visualizar uma amostra das colunas de data
df_convertido.select(colunas).show(10, truncate=False)


In [6]:
from pyspark.sql.functions import col, to_date

def detectar_e_converter_datas(df, formatos_tentados=None, max_amostras=None):
    """
    Detecta colunas string que parecem datas e converte para date.
    """
    # Define formatos padrão caso nenhum seja passado
    if formatos_tentados is None:
        formatos_tentados = [
            "yyyy-MM-dd",
            "dd/MM/yyyy",   # formato comum no Brasil
            "dd-MM-yyyy",
            "yyyy/MM/dd",
            "MM/dd/yyyy",
            "yyyy-MM",      # ano e mês apenas
            "yyyyMMdd",     # formato compacto
            "ddMMyyyy",
        ]
    # Define quantidade máxima de amostras por coluna
    if max_amostras is None:
        max_amostras = 100

    colunas_convertidas = []

    # Cache do schema (tipos de colunas)
    schema = dict(df.dtypes)

    for c in df.columns:
        # Só analisa colunas string
        if schema.get(c) != "string":
            continue

        # Coleta amostras não nulas da coluna
        try:
            amostras = (
                df.select(c)
                  .where(col(c).isNotNull())
                  .limit(max_amostras)
                  .toPandas()[c]
                  .dropna()
                  .astype(str)    # garante string
                  .str.strip()    # remove espaços extras
                  .tolist()
            )
        except Exception:
            amostras = []

        if not amostras:
            continue

        # Função auxiliar: remove parte de horário (ex.: "2024-01-01 12:00:00")
        def _limpo(v: str) -> str:
            if " " in v:
                v = v.split(" ", 1)[0]
            if "T" in v:
                v = v.split("T", 1)[0]
            return v

        amostras = [_limpo(v) for v in amostras if v]

        # Testa cada formato da lista
        for fmt in formatos_tentados:
            convertidos_validos = 0
            # Ajusta formato Spark -> formato Python
            py_fmt = fmt.replace("yyyy", "%Y").replace("MM", "%m").replace("dd", "%d")

            for valor in amostras:
                v = valor
                try:
                    # Caso especial: formato "yyyy-MM" precisa de dia para validar
                    if fmt == "yyyy-MM":
                        dt = datetime.strptime(v + "-01", "%Y-%m-%d")
                    else:
                        dt = datetime.strptime(v, py_fmt)

                    # Validação de faixa (anos plausíveis)
                    if 1950 <= dt.year <= 2050 and 1 <= dt.month <= 12 and 1 <= dt.day <= 31:
                        convertidos_validos += 1
                except Exception:
                    continue

            # Converte se >=70% das amostras forem válidas
            if convertidos_validos >= int(0.7 * len(amostras)):
                print(f"✅ Coluna '{c}' convertida com formato: {fmt}")
                df = df.withColumn(c, to_date(col(c), fmt))  # conversão Spark
                colunas_convertidas.append(c)
                break  # para de testar formatos após o primeiro sucesso

    return df, colunas_convertidas


### 📌 Função `comparar_datasets_parquet(path_a, path_b)`

Esta função tem como objetivo **comparar dois datasets em formato Parquet** e verificar se eles possuem **as mesmas colunas** e **os mesmos tipos de dados**, ignorando a ordem das colunas.  
Ela é útil em pipelines de ingestão e validação, quando é necessário garantir consistência de schema entre arquivos provenientes de diferentes fontes ou partições.

---

#### 🔎 O que a função faz

1. **Parâmetros**
   - `path_a`: caminho (string) do primeiro dataset Parquet.  
     - Pode incluir curingas (`*`) para ler múltiplos arquivos de uma vez.  
   - `path_b`: caminho (string) do segundo dataset Parquet.  
     - Também aceita curingas.

2. **Leitura dos datasets**
   - Usa o `spark.read.parquet` para carregar os dois arquivos em DataFrames Spark (`dfA` e `dfB`).  
   - Extrai o schema de cada DataFrame em formato de dicionário (`{coluna: tipo}`) por meio da função auxiliar `schema_dict`.

3. **Comparação de colunas**
   - Identifica colunas presentes em `dfA` mas ausentes em `dfB`, e vice-versa.  
   - Caso haja diferenças, imprime as colunas faltantes em cada dataset.

4. **Comparação de tipos**
   - Considera apenas as colunas comuns a ambos os datasets.  
   - Verifica se os tipos são idênticos (`string`, `int`, `double`, etc.).  
   - Caso alguma coluna tenha tipos diferentes entre A e B, imprime a divergência encontrada.

5. **Critério de igualdade**
   - Se **não houver colunas faltantes/extras** e **não houver divergências de tipo**, a função considera os datasets equivalentes em termos de schema.  
   - Retorna `True` nesse caso, caso contrário retorna `False`.

---

#### ✅ Exemplo de uso

```python
caminho_base = r"C:\Users\fred\meu_projeto_etl\data\raw\train"

# Comparar duas partições do mesmo grupo
resultado = comparar_datasets_parquet(
    fr"{caminho_base}\train_applprev_1_0.parquet",
    fr"{caminho_base}\train_applprev_1_1.parquet"
)

print("Resultado da comparação:", resultado)


In [47]:
# --- Verificar colunas e tipos entre dois datasets Parquet (ordem ignorada) ---

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CompararSchemas").getOrCreate()

def schema_dict(df):
    # ex.: {"case_id": "string", "valor": "double", ...}
    return {nome: tipo for nome, tipo in df.dtypes}

def comparar_datasets_parquet(path_a: str, path_b: str) -> bool:
    """
    Lê dois caminhos Parquet (podem aceitar curingas, ex.: ...\\tabela_*.parquet)
    e compara colunas e tipos (ordem ignorada).
    Retorna True se forem iguais, False caso contrário.
    """
    dfA = spark.read.parquet(path_a)
    dfB = spark.read.parquet(path_b)

    schA = schema_dict(dfA)
    schB = schema_dict(dfB)

    colsA, colsB = set(schA.keys()), set(schB.keys())

    ok = True

    # 1) Colunas faltantes/extras
    faltando_em_B = sorted(colsA - colsB)
    faltando_em_A = sorted(colsB - colsA)

    if faltando_em_B:
        ok = False
        print(f"🧩 Colunas presentes só em A (faltam em B): {faltando_em_B}")
    if faltando_em_A:
        ok = False
        print(f"🧩 Colunas presentes só em B (faltam em A): {faltando_em_A}")

    # 2) Tipos divergentes nas colunas comuns
    comuns = sorted(colsA & colsB)
    divergentes = [(c, schA[c], schB[c]) for c in comuns if schA[c] != schB[c]]
    if divergentes:
        ok = False
        print("⚠️ Colunas com tipos diferentes:")
        for c, ta, tb in divergentes:
            print(f"   - {c}: A={ta} | B={tb}")

    if ok:
        print("✅ Mesmas colunas e mesmos tipos (ordem ignorada).")
    return ok



## 1. 🧱 Tratamento da Tabela Base

In [7]:
#Importação da tabela base
test_base = spark.read.parquet(fr"{caminho}\\data\raw\test\test_base.parquet")

### Tratamento de nulos e duplicidade

In [8]:
# Função avaliar_dataset
df = test_base
nome_dataset = "test_base"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
✅ Chave primária é única.


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.0,0,10,"[57543, 57549, 57569]",
1,date_decision,string,0.0,0,9,"[2022-01-17, 2022-06-04, 2021-03-16]",
2,MONTH,bigint,0.0,0,1,[202201],
3,WEEK_NUM,bigint,0.0,0,1,[100],


{'total_registros': 10, 'colunas_com_nulos': 0, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 0}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\test_base_avaliar.csv


### Tratamento de tipos de dados

In [9]:
# Alteração de tipo da coluna date_decision
test_base_date = test_base.withColumn('date_decision', to_date(col('date_decision'), 'yyyy-MM-dd'))

In [10]:
# Verificação da alteração do tipo
test_base_date.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- date_decision: date (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- WEEK_NUM: long (nullable = true)



In [11]:
# Alteração de tipo da coluna MONTH
test_base_date = test_base_date.withColumn(
    "MONTH",
    to_date(format_string("%06d", col("MONTH")), "yyyyMM")
)

In [12]:
test_base_date.select('MONTH').distinct().orderBy('MONTH').show(20, truncate=False)

+----------+
|MONTH     |
+----------+
|2022-01-01|
+----------+



In [13]:
# Verificação da alteração do tipo
test_base_date.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- date_decision: date (nullable = true)
 |-- MONTH: date (nullable = true)
 |-- WEEK_NUM: long (nullable = true)



In [14]:
# Monta o caminho completo até o diretório de saída
caminho_saida = os.path.join(caminho, "data", "interim", "test_base_tratada")

# Salva o DataFrame Spark em formato Parquet no local desejado
test_base_date.write.mode("overwrite").parquet(caminho_saida)


### 📌 Resumo do tratamento da tabela *Base*

- ✅ Após análise e verificação de **unicidade**, a coluna **`case_id`** foi definida como **chave primária**.  
- ✅ As **colunas de datas** foram identificadas e convertidas para o tipo **`date`**.  
- ✅ A tabela tratada foi salva em formato **Parquet**.


## 2. 🛰️  Tratamento das tabelas "Satélite"

### Tabela de descrição de variáveis

## Empilhamento das tabelas

In [23]:
# Caminho base dos arquivos
caminho_1 = r"C:\Users\fred\meu_projeto_etl\data\raw\test"
caminho_base = caminho_1

# Lista dos grupos esperados
grupos = [
    "test_applprev_1",
    "test_applprev_2",
    "test_credit_bureau_a_1",
    "test_credit_bureau_a_2",
    "test_credit_bureau_b_1",
    "test_credit_bureau_b_2",
    "test_static_0",
    "test_base",
    "test_debitcard_1",
    "test_deposit_1",
    "test_other_1",
    "test_person_1",
    "test_person_2",
    "test_static_cb_0",
    "test_tax_registry_a_1",
    "test_tax_registry_b_1",
    "test_tax_registry_c_1"
]


In [24]:
import os, glob, re
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col

def _dtype_map(df):
    return {f.name: f.dataType.simpleString() for f in df.schema.fields}

def _print_coluna_problemática(e_msg, dfA, dfB):
    colsA, colsB = dfA.columns, dfB.columns
    tiposA, tiposB = _dtype_map(dfA), _dtype_map(dfB)

    m = re.search(r"(\d+)(?:st|nd|rd|th) column", e_msg)
    if m:
        idx = int(m.group(1)) - 1
        nomeA = colsA[idx] if idx < len(colsA) else None
        nomeB = colsB[idx] if idx < len(colsB) else None
        if nomeA == nomeB:
            return [nomeA]

    comuns = sorted(set(colsA).intersection(colsB))
    diverg = [c for c in comuns if tiposA.get(c) != tiposB.get(c)]
    return diverg

def carregar_e_empilhar(grupo):
    padrao = os.path.join(caminho_base, f"{grupo}_*.parquet")
    arquivos = sorted(glob.glob(padrao))
    arquivo_unico = os.path.join(caminho_base, f"{grupo}.parquet")
    if os.path.exists(arquivo_unico):
        arquivos.append(arquivo_unico)
    if not arquivos:
        print(f"🚫 Nenhum arquivo encontrado para {grupo}")
        return None

    colunas_padrao = None
    dfs_validos, paths_validos = [], []

    for caminho_arquivo in arquivos:
        df = spark.read.parquet(caminho_arquivo)
        if colunas_padrao is None:
            colunas_padrao = df.columns
            dfs_validos.append(df.select(colunas_padrao))
            paths_validos.append(caminho_arquivo)
            continue

        set_padrao, set_atual = set(colunas_padrao), set(df.columns)
        if set_padrao != set_atual:
            faltando = sorted(set_padrao - set_atual)
            extras = sorted(set_atual - set_padrao)
            print(f"🚫 Colunas diferentes em: {caminho_arquivo}")
            if faltando:
                print(f"   FALTANDO: {faltando}")
            if extras:
                print(f"   EXTRAS  : {extras}")
            continue

        df = df.select(colunas_padrao)
        dfs_validos.append(df)
        paths_validos.append(caminho_arquivo)

    if not dfs_validos:
        print(f"⚠️ Nenhum DataFrame válido para {grupo}")
        return None

    df_final = dfs_validos[0]
    for i in range(1, len(dfs_validos)):
        df_i = dfs_validos[i]
        try:
            df_final = df_final.unionByName(df_i)
        except AnalysisException as e:
            print("\n⛔ Tipos incompatíveis detectados durante o union.")
            colunas_problema = _print_coluna_problemática(str(e), df_final, df_i)
            print(f"🔄 Convertendo colunas para string: {colunas_problema}")

            for col_name in colunas_problema:
                df_final = df_final.withColumn(col_name, col(col_name).cast("string"))
                df_i = df_i.withColumn(col_name, col(col_name).cast("string"))

            try:
                df_final = df_final.unionByName(df_i)
                print(f"✅ Union refeito com cast para string nas colunas: {colunas_problema}")
            except Exception as e2:
                print(f"❌ Falha mesmo após cast: {e2}")
                return None

    print(f"✅ {grupo} carregado com {len(dfs_validos)} arquivo(s).")
    return df_final


In [25]:
# Dicionário com os DataFrames carregados com sucesso
variaveis_criadas = {}

# Carregar todos os grupos
for grupo in grupos:
    df = carregar_e_empilhar(grupo)
    if df is not None:
        variaveis_criadas[grupo] = df

# Exibir nomes dos DataFrames criados
print("\n📦 Variáveis carregadas:")
for nome in variaveis_criadas:
    print(f" - {nome}")


⛔ Tipos incompatíveis detectados durante o union.
🔄 Convertendo colunas para string: ['isdebitcard_527L']
✅ Union refeito com cast para string nas colunas: ['isdebitcard_527L']

⛔ Tipos incompatíveis detectados durante o union.
🔄 Convertendo colunas para string: ['isdebitcard_527L']
✅ Union refeito com cast para string nas colunas: ['isdebitcard_527L']
✅ test_applprev_1 carregado com 3 arquivo(s).
✅ test_applprev_2 carregado com 1 arquivo(s).
✅ test_credit_bureau_a_1 carregado com 5 arquivo(s).
✅ test_credit_bureau_a_2 carregado com 12 arquivo(s).
✅ test_credit_bureau_b_1 carregado com 1 arquivo(s).
✅ test_credit_bureau_b_2 carregado com 1 arquivo(s).

⛔ Tipos incompatíveis detectados durante o union.
🔄 Convertendo colunas para string: ['isbidproductrequest_292L']
✅ Union refeito com cast para string nas colunas: ['isbidproductrequest_292L']
✅ test_static_0 carregado com 3 arquivo(s).
✅ test_base carregado com 1 arquivo(s).
✅ test_debitcard_1 carregado com 1 arquivo(s).
✅ test_deposit

In [28]:
from pyspark.sql import DataFrame

def publicar_como_variaveis_globais(dfs_dict):
    for nome, df in dfs_dict.items():
        if isinstance(df, DataFrame):
            globals()[nome] = df
            print(f"↗️ publicado: {nome}")

publicar_como_variaveis_globais(variaveis_criadas)

# agora você pode usar diretamente:
# test_applprev_1.printSchema()


↗️ publicado: test_applprev_1
↗️ publicado: test_applprev_2
↗️ publicado: test_credit_bureau_a_1
↗️ publicado: test_credit_bureau_a_2
↗️ publicado: test_credit_bureau_b_1
↗️ publicado: test_credit_bureau_b_2
↗️ publicado: test_static_0
↗️ publicado: test_base
↗️ publicado: test_debitcard_1
↗️ publicado: test_deposit_1
↗️ publicado: test_other_1
↗️ publicado: test_person_1
↗️ publicado: test_person_2
↗️ publicado: test_static_cb_0
↗️ publicado: test_tax_registry_a_1
↗️ publicado: test_tax_registry_b_1
↗️ publicado: test_tax_registry_c_1


## Tratamento da tabela test_applprev_1_0

In [57]:
# Função avaliar_dataset
df = test_applprev_1
nome_dataset = "test_applprev_1"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 4 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,4,"[57543, 57549, 57760]",
1,actualdpd_943P,double,0.00,0,1,[0.0],
2,annuity_853A,double,0.00,0,22,"[3725.4001, 935.2, 4516.8003]",
3,approvaldate_319D,date,36.67,11,17,"[2016-08-31, 2011-12-24, 2018-12-29]",
4,byoccupationinc_3656910L,double,73.33,22,6,"[1.0, 15000.0, 35000.0]",
5,cancelreason_3545846M,string,0.00,0,3,"[a55475b1, P94_109_143, f2052b23]",
6,childnum_21L,int,0.00,0,3,"[1, 2, 0]",
7,creationdate_885D,date,0.00,0,24,"[2016-08-31, 2011-12-24, 2018-12-29]",
8,credacc_actualbalance_314A,double,83.33,25,5,"[34066.0, 17800.0, 0.0]",
9,credacc_credlmt_575A,double,0.00,0,6,"[0.0, 34066.0, 20000.0]",


{'total_registros': 30, 'colunas_com_nulos': 23, 'colunas_totalmente_nulas': 1, 'duplicatas_pk': 4}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\test_applprev_1_avaliar.csv


In [32]:
# Converter todas as colunas de data para DateType (assumindo formato yyyy-MM-dd)
test_applprev_1_date = test_applprev_1.withColumn("approvaldate_319D", to_date("approvaldate_319D")) \
       .withColumn("creationdate_885D", to_date("creationdate_885D")) \
       .withColumn("dateactivated_425D", to_date("dateactivated_425D")) \
       .withColumn("dtlastpmt_581D", to_date("dtlastpmt_581D")) \
       .withColumn("dtlastpmtallstes_3545839D", to_date("dtlastpmtallstes_3545839D")) \
       .withColumn("employedfrom_700D", to_date("employedfrom_700D")) \
       .withColumn("firstnonzeroinstldate_307D", to_date("firstnonzeroinstldate_307D"))


In [33]:
# Dias contados
test_applprev_1 = test_applprev_1_date.withColumn("dias_para_aprovacao", datediff("approvaldate_319D", "creationdate_885D")) \
       .withColumn("dias_ate_ativacao", datediff("dateactivated_425D", "creationdate_885D")) \
       .withColumn("dias_ult_pagamento", datediff("dtlastpmt_581D", "creationdate_885D")) \
       .withColumn("dias_ult_pagamento_all", datediff("dtlastpmtallstes_3545839D", "creationdate_885D")) \
       .withColumn("dias_desde_inicio_emprego", datediff("creationdate_885D", "employedfrom_700D")) \
       .withColumn("dias_para_primeira_parcela", datediff("firstnonzeroinstldate_307D", "creationdate_885D"))


In [34]:
# Criação de flags
test_applprev_1 = test_applprev_1 \
  .withColumn("sem_aprovacao_flag", when(col("approvaldate_319D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_ativacao_flag", when(col("dateactivated_425D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_pagamento_flag", when(col("dtlastpmt_581D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_pagamento_total_flag", when(col("dtlastpmtallstes_3545839D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_emprego_flag", when(col("employedfrom_700D").isNull(), 1).otherwise(0)) \
  .withColumn("sem_parcela_flag", when(col("firstnonzeroinstldate_307D").isNull(), 1).otherwise(0))


In [ ]:
colunas_float = [
    "credamount_590A",
    "currdebt_94A",
    "outstandingdebt_522A",
    "mainoccupationinc_437A",
    "byoccupationinc_3656910L",
    "annuity_853A",
    "revolvingaccount_394A"
]

for c in colunas_float:
    test_applprev_1 = test_applprev_1.withColumn(c, col(c).cast("double"))


In [35]:
# Flags de nulo/cruzamento
test_applprev_1 = test_applprev_1 \
    .withColumn("mainoccupationinc_null_flag", when(col("mainoccupationinc_437A").isNull(), 1).otherwise(0)) \
    .withColumn("byoccupationinc_null_flag", when(col("byoccupationinc_3656910L").isNull(), 1).otherwise(0)) \
    .withColumn("tem_revolving_flag", when(col("revolvingaccount_394A").isNotNull(), 1).otherwise(0))



In [36]:
from pyspark.sql.functions import coalesce

test_applprev_1 = test_applprev_1.withColumn("divida_total",
                   coalesce(col("currdebt_94A"), lit(0)) + coalesce(col("outstandingdebt_522A"), lit(0)))


In [37]:
# Criação de flag
test_applprev_1 = test_applprev_1 \
  .withColumn("limite_cartao_credito_flag", when(col("approvaldate_319D").isNull(), 1).otherwise(0))

In [38]:
from pyspark.sql.functions import when

categoricas_com_nulos = [
    "credtype_587L",
    "familystate_726L",
    "inittransactioncode_279L"
]

for c in categoricas_com_nulos:
    test_applprev_1 = test_applprev_1.withColumn(
        c,
        when(col(c).isNull(), "Desconhecido").otherwise(col(c))
    )


In [39]:
from pyspark.sql.types import StringType

categoricas_todas = [
    "cancelreason_3545846M", "credtype_587L", "district_544M",
    "education_1138M", "familystate_726L", "inittransactioncode_279L",
    "postype_4733339M", "profession_152M", "rejectreason_755M", "rejectreasonclient_4145042M"
]

for c in categoricas_todas:
    test_applprev_1 = test_applprev_1.withColumn(c, col(c).cast(StringType()))


In [40]:
from pyspark.sql.functions import when, col

# 🔹 1. Tratamento de flags booleanas (transformar em 0/1)
test_applprev_1 = test_applprev_1 \
    .withColumn("isbidproduct_390L", when(col("isbidproduct_390L") == True, 1)
                                      .when(col("isbidproduct_390L") == False, 0)
                                      .otherwise(0)) \
    .withColumn("isdebitcard_527L", when(col("isdebitcard_527L") == True, 1)
                                     .when(col("isdebitcard_527L") == False, 0)
                                     .otherwise(0))

# 🔹 2. Tratamento de colunas de contagem (nulos → 0, cast para int)
cols_contagem = [
    "childnum_21L",
    "pmtnum_8L",
    "tenor_203L",
    "credacc_transactions_402L"
]

for c in cols_contagem:
    test_applprev_1 = test_applprev_1.withColumn(
        c,
        (when(col(c).isNull(), 0).otherwise(col(c))).cast("int")
    )

# 🔹 3. Tratamento de categorias discretas com valor numérico (nulos → -1, cast para int)
test_applprev_1 = test_applprev_1 \
    .withColumn("credacc_status_367L", (when(col("credacc_status_367L").isNull(), -1)
                                        .otherwise(col("credacc_status_367L"))).cast("int")) \
    .withColumn("status_219L", (when(col("status_219L").isNull(), -1)
                                .otherwise(col("status_219L"))).cast("int"))


In [41]:
from pyspark.sql.functions import col, when, lit, first, avg, collect_set

from pyspark.sql.functions import when, col

# Define as colunas que indicam histórico de crédito anterior
colunas_credito = [
    "credacc_actualbalance_314A",
    "credacc_maxhisbal_375A",
    "credacc_minhisbal_90A",
    "downpmt_134A",
    "maxdpdtolerance_577P"
]

# Cria uma condição para verificar se TODAS estão nulas
condicao_sem_credito = None
for c in colunas_credito:
    condicao_sem_credito = col(c).isNull() if condicao_sem_credito is None else (condicao_sem_credito & col(c).isNull())

# Cria a flag binária
test_applprev_1 = test_applprev_1.withColumn(
    "sem_historico_credito_flag",
    when(condicao_sem_credito, 1).otherwise(0)
)

In [42]:
# Categóricas: preenchimento de nulo e cast para string
colunas_categoricas = [
    "district_544M",
    "education_1138M",
    "profession_152M",
    "postype_4733339M",
    "rejectreason_755M",
    "rejectreasonclient_4145042M"
]

for c in colunas_categoricas:
    test_applprev_1 = test_applprev_1.withColumn(c, when(col(c).isNull(), "Desconhecido").otherwise(col(c)).cast("string"))

In [61]:
test_applprev_1.coalesce(1).write.mode("overwrite").parquet(fr"{caminho}\\data\interim\test_applprev_1.parquet")

In [62]:
caminho_base = r"C:\Users\fred\meu_projeto_etl\data\interim"
comparar_datasets_parquet(fr"{caminho_base}\\train_applprev_1.parquet",
                          fr"{caminho_base}\\test_applprev_1.parquet")


✅ Mesmas colunas e mesmos tipos (ordem ignorada).


True

### 📌 Resumo do tratamento da tabela *test_applprev_1*

- ✅ Os **datasets particionados** foram carregados em uma única variável, permitindo que o tratamento fosse feito de forma **unificada**.  
- ✅ A função de **avaliação** foi executada para observar o **comportamento e formato** dos dados.  
- ✅ As **colunas que apresentaram formato de data** foram convertidas para o tipo **`date`**.  
- ✅ Foi criada uma **coluna de tempo (em dias)** para comparação entre **datas iniciais e finais**.  
- ✅ Foram criadas **flags** para indicar a presença de **valores nulos**.  
- ✅ Foi criada uma coluna de **dívida total**.  
- ✅ Colunas que tinham valores **numéricos mas eram categóricas** foram convertidas para **string**.  
- ✅ A tabela tratada foi salva em formato **Parquet**.
- ✅ A comparação com o dataset **train** correspondente foi realizada **confirmando** a relação.

## Tratamento da tabela train_applprev_2

In [63]:
# Função avaliar_dataset
df = test_applprev_2
nome_dataset = "test_applprev_2"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")



📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 1 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.0,0,1,[57543],
1,cacccardblochreas_147M,string,0.0,0,1,[a55475b1],
2,conts_type_509L,string,40.0,4,3,"[PRIMARY_MOBILE, PHONE]",
3,credacc_cards_status_52L,int,100.0,10,1,[],
4,num_group1,bigint,0.0,0,5,"[0, 1, 3]",
5,num_group2,bigint,0.0,0,2,"[0, 1]",


{'total_registros': 10, 'colunas_com_nulos': 2, 'colunas_totalmente_nulas': 1, 'duplicatas_pk': 1}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\test_applprev_2_avaliar.csv


### 📌 Resumo do tratamento da tabela *train_applprev_2*

- ✅ A tabela foi **avaliada** com a função de profiling (`avaliar_dataset`).  
- ❌ **Não foi incorporada ao pipeline de tratamento**, devido à:  
  - Baixa relevância das informações.  
  - Impossibilidade de definição de uma chave primária.  


## Tratamento da tabela train_credit_bureau_a_1

In [64]:
# Função avaliar_dataset
df = test_credit_bureau_a_1
nome_dataset = "test_credit_bureau_a_1"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")


📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 5 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.0,0,5,"[57551, 57543, 57633]",
1,annualeffectiverate_199L,double,74.0,37,8,"[8.28, 0.69, 34.5]",
2,annualeffectiverate_63L,double,80.0,40,6,"[34.5, 56.0, 38.48]",
3,classificationofcontr_13M,string,0.0,0,2,"[ea6782cc, a55475b1]",
4,classificationofcontr_400M,string,0.0,0,8,"[e6e56e83, ea6782cc, ada1729d]",
...,...,...,...,...,...,...,...
140,totalamount_996A_flag,int,0.0,0,2,"[1, 0]",
141,totaldebtoverduevalue_178A_flag,int,0.0,0,2,"[1, 0]",
142,totaldebtoverduevalue_718A_flag,int,0.0,0,2,"[1, 0]",
143,totaloutstanddebtvalue_39A_flag,int,0.0,0,2,"[1, 0]",


{'total_registros': 50, 'colunas_com_nulos': 66, 'colunas_totalmente_nulas': 2, 'duplicatas_pk': 5}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\test_credit_bureau_a_1_avaliar.csv


In [52]:
# Executar a função criar_flags_nulos
test_credit_bureau_a_1_flags = criar_flags_nulos(test_credit_bureau_a_1)

# Listar todas as flags criadas
flags_criadas = [c for c in test_credit_bureau_a_1_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
test_credit_bureau_a_1_flags.select(flags_criadas).limit(5).toPandas()

🟩 Sem valores ausentes em: case_id (flag não criada)
✅ Flag criada: annualeffectiverate_199L_flag
✅ Flag criada: annualeffectiverate_63L_flag
🟩 Sem valores ausentes em: classificationofcontr_13M (flag não criada)
🟩 Sem valores ausentes em: classificationofcontr_400M (flag não criada)
🟩 Sem valores ausentes em: contractst_545M (flag não criada)
🟩 Sem valores ausentes em: contractst_964M (flag não criada)
✅ Flag criada: contractsum_5085717L_flag
✅ Flag criada: credlmt_230A_flag
✅ Flag criada: credlmt_935A_flag
✅ Flag criada: dateofcredend_289D_flag
✅ Flag criada: dateofcredend_353D_flag
✅ Flag criada: dateofcredstart_181D_flag
✅ Flag criada: dateofcredstart_739D_flag
✅ Flag criada: dateofrealrepmt_138D_flag
✅ Flag criada: debtoutstand_525A_flag
✅ Flag criada: debtoverdue_47A_flag
🟩 Sem valores ausentes em: description_351M (flag não criada)
✅ Flag criada: dpdmax_139P_flag
✅ Flag criada: dpdmax_757P_flag
✅ Flag criada: dpdmaxdatemonth_442T_flag
✅ Flag criada: dpdmaxdatemonth_89T_flag
✅ Fl

,annualeffectiverate_199L_flag,annualeffectiverate_63L_flag,contractsum_5085717L_flag,credlmt_230A_flag,credlmt_935A_flag,dateofcredend_289D_flag,dateofcredend_353D_flag,dateofcredstart_181D_flag,dateofcredstart_739D_flag,dateofrealrepmt_138D_flag,...,prolongationcount_599L_flag,refreshdate_3813885D_flag,residualamount_488A_flag,residualamount_856A_flag,totalamount_6A_flag,totalamount_996A_flag,totaldebtoverduevalue_178A_flag,totaldebtoverduevalue_718A_flag,totaloutstanddebtvalue_39A_flag,totaloutstanddebtvalue_668A_flag
0,0,1,1,1,1,1,0,0,1,0,...,1,1,1,1,0,1,1,1,1,1
1,1,1,0,1,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
2,1,1,0,1,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,1
3,1,1,1,0,1,1,0,0,1,0,...,1,1,0,1,1,1,1,1,1,1
4,1,1,1,0,1,1,0,0,1,0,...,1,1,0,1,1,1,1,1,1,1


In [53]:
# Executar a função detectar_e_converter_datas
test_credit_bureau_a_1_date, cols_conv = detectar_e_converter_datas(test_credit_bureau_a_1_flags)

print("Colunas convertidas:", cols_conv)

# Verificar schema após conversão
print(test_credit_bureau_a_1_date.printSchema())

# Exibir amostra das colunas convertidas (sem coletar tudo)
test_credit_bureau_a_1_date.select(*cols_conv).show(10, truncate=False)

✅ Coluna 'dateofcredend_289D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofcredend_353D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofcredstart_181D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofcredstart_739D' convertida com formato: yyyy-MM-dd
✅ Coluna 'dateofrealrepmt_138D' convertida com formato: yyyy-MM-dd
✅ Coluna 'lastupdate_1112D' convertida com formato: yyyy-MM-dd
✅ Coluna 'lastupdate_388D' convertida com formato: yyyy-MM-dd
✅ Coluna 'numberofoverdueinstlmaxdat_148D' convertida com formato: yyyy-MM-dd
✅ Coluna 'numberofoverdueinstlmaxdat_641D' convertida com formato: yyyy-MM-dd
✅ Coluna 'overdueamountmax2date_1002D' convertida com formato: yyyy-MM-dd
✅ Coluna 'overdueamountmax2date_1142D' convertida com formato: yyyy-MM-dd
✅ Coluna 'refreshdate_3813885D' convertida com formato: yyyy-MM-dd
Colunas convertidas: ['dateofcredend_289D', 'dateofcredend_353D', 'dateofcredstart_181D', 'dateofcredstart_739D', 'dateofrealrepmt_138D', 'lastupdate_1112D', 'lastupdate_388

+------------------+------------------+--------------------+--------------------+--------------------+----------------+---------------+-------------------------------+-------------------------------+---------------------------+---------------------------+--------------------+
|dateofcredend_289D|dateofcredend_353D|dateofcredstart_181D|dateofcredstart_739D|dateofrealrepmt_138D|lastupdate_1112D|lastupdate_388D|numberofoverdueinstlmaxdat_148D|numberofoverdueinstlmaxdat_641D|overdueamountmax2date_1002D|overdueamountmax2date_1142D|refreshdate_3813885D|
+------------------+------------------+--------------------+--------------------+--------------------+----------------+---------------+-------------------------------+-------------------------------+---------------------------+---------------------------+--------------------+
|NULL              |2023-07-02        |2018-09-10          |NULL                |2020-06-20          |NULL            |2020-06-24     |2018-11-04                     |NU

In [83]:
test_credit_bureau_a_1_group = test_credit_bureau_a_1_date.withColumn('num_group1', col('num_group1').cast('bigint'))

In [86]:
test_credit_bureau_a_1 = test_credit_bureau_a_1_group
test_credit_bureau_a_1.coalesce(1).write.mode("overwrite").parquet(
    fr"{caminho}\\data\interim\test_bureau_a_1.parquet"
)

In [87]:
caminho_base = r"C:\Users\fred\meu_projeto_etl\data\interim"
comparar_datasets_parquet(fr"{caminho_base}\\train_bureau_a_1.parquet",
                          fr"{caminho_base}\\test_bureau_a_1.parquet")

✅ Mesmas colunas e mesmos tipos (ordem ignorada).


True

### 📌 Resumo do tratamento da tabela *train_credit_bureau_a_1*

- ✅ A função de **avaliação do dataset** foi executada.  
- ✅ A função de **criação de flags de nulos** foi aplicada, gerando diversas novas colunas.  
- ✅ A função de **detecção e conversão de datas** foi executada, convertendo várias colunas para o tipo `date`.
- ✅ A comparação com o dataset **train** correspondente foi realizada e a coluna **num_group1** e a mesma foi **tratada**.
- 🔎 Após avaliação detalhada das colunas e seus valores, **não foram necessárias novas alterações**.  
- 💾 O dataset tratado foi salvo em formato **Parquet**.  


## Tratamento da tabela train_credit_bureau_a_2

In [90]:
# Função avaliar_dataset
df = test_credit_bureau_a_2
nome_dataset = "test_credit_bureau_a_2"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")

📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 12 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.00,0,12,"[57551, 57631, 57681]",
1,collater_typofvalofguarant_298M,string,0.00,0,3,"[8fd95e4b, 9a0c095e, a55475b1]",
2,collater_typofvalofguarant_407M,string,0.00,0,3,"[8fd95e4b, 9a0c095e, a55475b1]",
3,collater_valueofguarantee_1124L,double,76.67,92,3,"[0.0, 7230000.0]",
4,collater_valueofguarantee_876L,double,44.17,53,3,"[0.0, 75200.0]",
5,collaterals_typeofguarante_359M,string,0.00,0,3,"[c7a5ad39, 3cbe86ba, a55475b1]",
6,collaterals_typeofguarante_669M,string,0.00,0,3,"[c7a5ad39, a55475b1, 7b62420e]",
7,num_group1,bigint,0.00,0,13,"[0, 7, 6]",
8,num_group2,bigint,0.00,0,10,"[0, 7, 6]",
9,pmts_dpd_1073P,double,84.17,101,2,[0.0],


{'total_registros': 120, 'colunas_com_nulos': 10, 'colunas_totalmente_nulas': 0, 'duplicatas_pk': 12}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\test_credit_bureau_a_2_avaliar.csv


In [91]:
# Executar a função criar_flags_nulos
test_credit_bureau_a_2_flags = criar_flags_nulos(test_credit_bureau_a_2)

# Listar todas as flags criadas
flags_criadas = [c for c in test_credit_bureau_a_2_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
test_credit_bureau_a_2_flags.select(flags_criadas).limit(5).toPandas()

🟩 Sem valores ausentes em: case_id (flag não criada)
🟩 Sem valores ausentes em: collater_typofvalofguarant_298M (flag não criada)
🟩 Sem valores ausentes em: collater_typofvalofguarant_407M (flag não criada)
✅ Flag criada: collater_valueofguarantee_1124L_flag
✅ Flag criada: collater_valueofguarantee_876L_flag
🟩 Sem valores ausentes em: collaterals_typeofguarante_359M (flag não criada)
🟩 Sem valores ausentes em: collaterals_typeofguarante_669M (flag não criada)
🟩 Sem valores ausentes em: num_group1 (flag não criada)
🟩 Sem valores ausentes em: num_group2 (flag não criada)
✅ Flag criada: pmts_dpd_1073P_flag
✅ Flag criada: pmts_dpd_303P_flag
✅ Flag criada: pmts_month_158T_flag
✅ Flag criada: pmts_month_706T_flag
✅ Flag criada: pmts_overdue_1140A_flag
✅ Flag criada: pmts_overdue_1152A_flag
✅ Flag criada: pmts_year_1139T_flag
✅ Flag criada: pmts_year_507T_flag
🟩 Sem valores ausentes em: subjectroles_name_541M (flag não criada)
🟩 Sem valores ausentes em: subjectroles_name_838M (flag não criada

,collater_valueofguarantee_1124L_flag,collater_valueofguarantee_876L_flag,pmts_dpd_1073P_flag,pmts_dpd_303P_flag,pmts_month_158T_flag,pmts_month_706T_flag,pmts_overdue_1140A_flag,pmts_overdue_1152A_flag,pmts_year_1139T_flag,pmts_year_507T_flag
0,0,0,1,1,0,0,1,1,0,0
1,0,0,1,1,0,0,1,1,0,0
2,1,0,1,1,1,0,1,1,1,0
3,1,0,1,1,1,0,1,1,1,0
4,1,0,1,1,1,0,1,1,1,0


In [92]:
test_credit_bureau_a_2 = test_credit_bureau_a_2_flags
test_credit_bureau_a_2.coalesce(1).write.mode("overwrite").parquet(
    fr"{caminho}\\data\interim\test_bureau_a_2.parquet"
)

In [93]:
caminho_base = r"C:\Users\fred\meu_projeto_etl\data\interim"
comparar_datasets_parquet(fr"{caminho_base}\\test_bureau_a_2.parquet",
                          fr"{caminho_base}\\test_bureau_a_2.parquet")

✅ Mesmas colunas e mesmos tipos (ordem ignorada).


True

### 📌 Resumo do tratamento da tabela *train_credit_bureau_a_2*

- ✅ Execução da função de **avaliação do dataset**.  
- ✅ **Avaliação visual** do dataset.  
- ✅ Execução da função de **criação de flags** (valores nulos).  
- ✅ A comparação com o dataset **train** correspondente foi realizada **confirmando** a relação.
- 💾 Dataset salvo em formato **Parquet**.  
  


## Tratamento da tabela train_credit_bureau_b_1

In [94]:
# Função avaliar_dataset
df = test_credit_bureau_b_1
nome_dataset = "test_credit_bureau_b_1"

# Avaliação
tabela_resultado, resumo = avaliar_dataset(df=df, pk="case_id")

# Visualização
from IPython.display import display
display(tabela_resultado)
print(resumo)

# Nome e caminho do arquivo
nome_arquivo = f"{nome_dataset}_avaliar"
caminho_saida = fr"{caminho}\\docs\\{nome_arquivo}.csv"

# Salvar CSV
tabela_resultado.to_csv(caminho_saida, index=False, encoding="utf-8-sig")
print(f"✅ Tabela salva em: {caminho_saida}")

📊 Avaliação do DataFrame...

🔍 Chave primária 'case_id':
⚠️ Encontradas 3 duplicatas com base na chave 'case_id'


,coluna,tipo,% nulos,qtd_nulos,valores_distintos,exemplo_valores,observacoes
0,case_id,bigint,0.0,0,3,"[57675, 57754, 57775]",
1,amount_1115A,double,60.0,6,5,"[1932619.4, 3000.0, 1488000.0]",
2,classificationofcontr_1114M,string,0.0,0,3,"[ea6782cc, a55475b1, 01f63ac8]",
3,contractdate_551D,string,0.0,0,10,"[2020-04-22, 2021-01-20, 2020-06-26]",
4,contractmaturitydate_151D,string,0.0,0,10,"[2020-07-25, 2035-04-23, 2022-09-29]",
5,contractst_516M,string,0.0,0,3,"[04bf6e27, 7241344e, 0dc85f9d]",
6,contracttype_653M,string,0.0,0,4,"[1c9c5356, 60e784d6, 4257cbed]",
7,credlmt_1052A,double,80.0,8,3,"[132032.0, 220598.0]",
8,credlmt_228A,double,80.0,8,3,"[10000.0, 38000.0]",
9,credlmt_3940954A,double,40.0,4,6,"[0.0, 35232.0, 200598.0]",


{'total_registros': 10, 'colunas_com_nulos': 22, 'colunas_totalmente_nulas': 1, 'duplicatas_pk': 3}
✅ Tabela salva em: C:\Users\fred\meu_projeto_etl\\docs\\test_credit_bureau_b_1_avaliar.csv


In [95]:
from pyspark.sql import functions as F

COL = "maxdebtpduevalodued_3940955A"  # coluna de dias
MAX_DIAS_PLAUSIVEL = 2000             # ajuste se necessário

# 1) Filtrar valores absurdos (negativos ou > limite)
absurdos_df = (
    test_credit_bureau_b_1
    .filter((F.col(COL) < 0) | (F.col(COL) > F.lit(MAX_DIAS_PLAUSIVEL)))
)

qtd_absurdos = absurdos_df.count()

if qtd_absurdos == 0:
    # 2) Converter para inteiro (arredondando)
    test_credit_bureau_b_1 = test_credit_bureau_b_1.withColumn(COL, F.round(F.col(COL)).cast("int"))
    print(f"✅ Sem valores absurdos detectados. Coluna '{COL}' convertida para int.")
else:
    print(f"⚠️ Encontrados {qtd_absurdos} valores absurdos em '{COL}'. Exemplos:")
    absurdos_df.select(COL).show(20, truncate=False)  # mostra até 20 exemplos


✅ Sem valores absurdos detectados. Coluna 'maxdebtpduevalodued_3940955A' convertida para int.


In [96]:
test_credit_bureau_b_1 = test_credit_bureau_b_1.withColumn(
    COL,
    F.when((F.col(COL) < 0) | (F.col(COL) > MAX_DIAS_PLAUSIVEL), None).otherwise(F.col(COL))
)
print("✅ Valores absurdos substituídos por nulo.")


✅ Valores absurdos substituídos por nulo.


In [97]:
from pyspark.sql import functions as F

COL = "maxdebtpduevalodued_3940955A"  # coluna de dias
MAX_DIAS_PLAUSIVEL = 2000             # ajuste se necessário

# 1) Filtrar valores absurdos (negativos ou > limite)
absurdos_df = (
    test_credit_bureau_b_1
    .filter((F.col(COL) < 0) | (F.col(COL) > F.lit(MAX_DIAS_PLAUSIVEL)))
)

qtd_absurdos = absurdos_df.count()

if qtd_absurdos == 0:
    # 2) Converter para inteiro (arredondando)
    test_credit_bureau_b_1 = test_credit_bureau_b_1.withColumn(COL, F.round(F.col(COL)).cast("int"))
    print(f"✅ Sem valores absurdos detectados. Coluna '{COL}' convertida para int.")
else:
    print(f"⚠️ Encontrados {qtd_absurdos} valores absurdos em '{COL}'. Exemplos:")
    absurdos_df.select(COL).show(20, truncate=False)  # mostra até 20 exemplos


✅ Sem valores absurdos detectados. Coluna 'maxdebtpduevalodued_3940955A' convertida para int.


In [98]:
from pyspark.sql import functions as F

COL = "overdueamountmaxdateyear_432T"

df_check = (
    test_credit_bureau_b_1
    .select(
        F.count(F.when(F.col(COL).isNull(), 1)).alias("qtd_nulos"),
        F.count(F.when(F.col(COL) == 1900, 1)).alias("qtd_1900")
    )
)

df_check.show(truncate=False)


+---------+--------+
|qtd_nulos|qtd_1900|
+---------+--------+
|0        |0       |
+---------+--------+



In [99]:
from pyspark.sql import functions as F

COL = "overdueamountmaxdateyear_432T"

anos_invalidos = (
    test_credit_bureau_b_1
    .filter((F.col(COL) < 1950) | (F.col(COL) > 2025))
    .groupBy(COL)
    .count()
    .orderBy("count", ascending=False)
)

anos_invalidos.show(truncate=False)


+-----------------------------+-----+
|overdueamountmaxdateyear_432T|count|
+-----------------------------+-----+
+-----------------------------+-----+



In [100]:
from pyspark.sql import functions as F

COL = "overdueamountmaxdateyear_432T"

test_credit_bureau_b_1 = test_credit_bureau_b_1.withColumn(
    COL,
    F.when(F.col(COL) == 1900, None).otherwise(F.col(COL))
)

print(f"✅ Valores 1900 em '{COL}' foram substituídos por NULL.")


✅ Valores 1900 em 'overdueamountmaxdateyear_432T' foram substituídos por NULL.


In [101]:
# Executar a função criar_flags_nulos
test_credit_bureau_b_1_flags = criar_flags_nulos(test_credit_bureau_b_1)

# Listar todas as flags criadas
flags_criadas = [c for c in test_credit_bureau_b_1_flags.columns if c.endswith("_flag")]
print("Flags criadas:", flags_criadas)

# Se quiser inspecionar algumas flags específicas:
test_credit_bureau_b_1_flags.select(flags_criadas).limit(5).toPandas()

🟩 Sem valores ausentes em: case_id (flag não criada)
✅ Flag criada: amount_1115A_flag
🟩 Sem valores ausentes em: classificationofcontr_1114M (flag não criada)
🟩 Sem valores ausentes em: contractdate_551D (flag não criada)
🟩 Sem valores ausentes em: contractmaturitydate_151D (flag não criada)
🟩 Sem valores ausentes em: contractst_516M (flag não criada)
🟩 Sem valores ausentes em: contracttype_653M (flag não criada)
✅ Flag criada: credlmt_1052A_flag
✅ Flag criada: credlmt_228A_flag
✅ Flag criada: credlmt_3940954A_flag
🟩 Sem valores ausentes em: credor_3940957M (flag não criada)
✅ Flag criada: credquantity_1099L_flag
✅ Flag criada: credquantity_984L_flag
🟩 Sem valores ausentes em: debtpastduevalue_732A (flag não criada)
✅ Flag criada: debtvalue_227A_flag
✅ Flag criada: dpd_550P_flag
✅ Flag criada: dpd_733P_flag
🟩 Sem valores ausentes em: dpdmax_851P (flag não criada)
🟩 Sem valores ausentes em: dpdmaxdatemonth_804T (flag não criada)
🟩 Sem valores ausentes em: dpdmaxdateyear_742T (flag não c

,amount_1115A_flag,credlmt_1052A_flag,credlmt_228A_flag,credlmt_3940954A_flag,credquantity_1099L_flag,credquantity_984L_flag,debtvalue_227A_flag,dpd_550P_flag,dpd_733P_flag,installmentamount_644A_flag,...,interesteffectiverate_369L_flag,interestrateyearly_538L_flag,numberofinstls_810L_flag,periodicityofpmts_997L_flag,pmtnumpending_403L_flag,residualamount_1093A_flag,residualamount_127A_flag,residualamount_3940956A_flag,totalamount_503A_flag,totalamount_881A_flag
0,0,1,1,1,1,1,0,1,1,1,...,1,1,0,1,0,1,1,1,1,1
1,1,0,0,0,0,0,1,0,0,0,...,1,1,1,1,1,0,0,0,0,0
2,1,1,1,0,0,0,1,0,0,0,...,1,1,1,1,1,1,1,0,0,0
3,0,1,1,1,1,1,0,1,1,1,...,1,1,0,1,0,1,1,1,1,1
4,1,0,0,0,0,0,1,0,0,0,...,1,1,1,1,1,0,0,0,0,0


In [102]:
# Executar a função detectar_e_converter_datas
test_credit_bureau_b_1_date, cols_conv = detectar_e_converter_datas(test_credit_bureau_b_1_flags)

print("Colunas convertidas:", cols_conv)

# Verificar schema após conversão
print(test_credit_bureau_b_1_date.printSchema())

# Exibir amostra das colunas convertidas (sem coletar tudo)
test_credit_bureau_b_1_date.select(*cols_conv).show(10, truncate=False)

✅ Coluna 'contractdate_551D' convertida com formato: yyyy-MM-dd
✅ Coluna 'contractmaturitydate_151D' convertida com formato: yyyy-MM-dd
✅ Coluna 'lastupdate_260D' convertida com formato: yyyy-MM-dd
Colunas convertidas: ['contractdate_551D', 'contractmaturitydate_151D', 'lastupdate_260D']
root
 |-- case_id: long (nullable = true)
 |-- amount_1115A: double (nullable = true)
 |-- classificationofcontr_1114M: string (nullable = true)
 |-- contractdate_551D: date (nullable = true)
 |-- contractmaturitydate_151D: date (nullable = true)
 |-- contractst_516M: string (nullable = true)
 |-- contracttype_653M: string (nullable = true)
 |-- credlmt_1052A: double (nullable = true)
 |-- credlmt_228A: double (nullable = true)
 |-- credlmt_3940954A: double (nullable = true)
 |-- credor_3940957M: string (nullable = true)
 |-- credquantity_1099L: double (nullable = true)
 |-- credquantity_984L: double (nullable = true)
 |-- debtpastduevalue_732A: double (nullable = true)
 |-- debtvalue_227A: double (nul

In [103]:
test_credit_bureau_b_1 = test_credit_bureau_b_1_date
test_credit_bureau_b_1.coalesce(1).write.mode("overwrite").parquet(
    fr"{caminho}\\data\interim\test_bureau_b_1.parquet"
)

In [104]:
caminho_base = r"C:\Users\fred\meu_projeto_etl\data\interim"
comparar_datasets_parquet(fr"{caminho_base}\\test_bureau_b_1.parquet",
                          fr"{caminho_base}\\test_bureau_b_1.parquet")

✅ Mesmas colunas e mesmos tipos (ordem ignorada).


True

### 📌 Resumo do tratamento da tabela *train_credit_bureau_b_1*

- ✅ Execução da função para **avaliar dataset**.  
- ✅ **Avaliação visual** do dataset.  
- ✅ **Correção de inconsistências** na coluna `maxdebtpduevalodued_3940955A`.  
- ✅ Execução da função para **criação de flags**.  
- ✅ Execução da função para **tratamento de datas**. 
- ✅ A comparação com o dataset **train** correspondente foi realizada **confirmando** a relação.
- 💾 Arquivo final salvo em formato **Parquet**.  
